In [3]:
import os
import datetime as dt
from clearml import Task


In [7]:
os.environ['CLEARML_API_HOST'] = "https://api.clear.ml"
os.environ['CLEARML_WEB_HOST'] = "https://app.clear.ml"
os.environ['CLEARML_FILES_HOST'] = "https://files.clear.ml"
os.environ['CLEARML_API_ACCESS_KEY'] = "NXRS736MXOJ50V3NIZD5"
os.environ['CLEARML_API_SECRET_KEY'] = "ibFYNMzbUiVb8VpRZOZducTBVRUHfpDMWLe4AQE2C4vrsnl4JU"

current_date = dt.datetime.now().strftime('%d-%m')
#task = Task.init(project_name="ArLab Classifier", task_name=f"3 : Train model {current_date}")

dataset_task = Task.get_task(project_name="ArLab Classifier", task_name=f"2 : Preprocess dataset {current_date}")
print(dataset_task.artifacts.keys())

2023-05-03 15:55:35,524 - clearml.util - WARNING - 6 task found when searching for `{'project_name': 'ArLab Classifier', 'task_name': '2 : Preprocess dataset 03-05', 'include_archived': True}`
2023-05-03 15:55:35,524 - clearml.util - WARNING - Selected task `2 : Preprocess dataset 03-05` (id=c289e64c1a7b47cd807ebf6b032a0f29)
dict_keys(['X_test', 'X_train', 'vectorizer', 'y_test', 'y_train'])


In [ ]:
https://files.clear.ml/ArLab%20Classifier/2%20%253A%20Preprocess%20dataset%2003-05.8ea36bd96b3546f991a879c9134ee82f/artifacts/y_test/y_test.pkl
https://files.clear.ml/ArLab%20Classifier/2%20%253A%20Preprocess%20dataset%2003-05.8ea36bd96b3546f991a879c9134ee82f/artifacts/y_test/y_test.pkl

In [2]:
from pydantic import BaseModel
import joblib

In [3]:


# Define input and output data types
class InputData(BaseModel):
    text: str

class OutputData(BaseModel):
    label: str
    confidence: float

def predict(input_data: InputData):
    # Process the input data
    text = input_data.text
    X = vectorizer.transform([text])

    # Pass the data through your model to get the prediction
    pred = model.predict(X)
    proba = model.predict_proba(X)[0][pred]

    # Return the prediction as output data
    return {"label": pred[0], "confidence": proba}

/Users/robinfournier/Desktop/arlab/arlabenv/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/robinfournier/Desktop/arlab/arlabenv/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
model = joblib.load("models/nb_03-05.pkl")
vectorizer = joblib.load("models/vectorizer.pkl")

/Users/robinfournier/Desktop/arlab/arlabenv/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/robinfournier/Desktop/arlab/arlabenv/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
text = "i don't know"
X = vectorizer.transform([text])

UnpicklingError: invalid load key, 'x'.

In [ ]:
api:
    build:
      context: ./api
      dockerfile: ./Dockerfile
    environment:
      <<: *airflow-common-env
    ports:
      - "8000:8000"
    volumes:
      - ./models:/app/models

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split



In [ ]:
df = pd.read_csv(f'datasets/full_articles_{current_date}.csv')

df = df.dropna()
X_train, X_test, y_train, y_test = train_test_split(df['article_text'], df['label'], test_size=0.2, random_state=42)
vectorizer = CountVectorizer(stop_words=custom_stopwords_fr())
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [2]:
import requests

In [3]:
api_url = "http://0.0.0.0:8000/predict"

response = requests.post(api_url, json={"text": "Je trouve très fort le joueur micheal jordan"})
    
    
    # Display the response
if response.status_code == 200:
    print(response.json())
else:
    print("Error:", response.status_code)

{'label': 'basket'}


In [6]:
response.json()['label']

'basket'

In [1]:
import pandas as pd

df = pd.read_csv('datasets/full_articles_04-05.csv')

In [2]:
df

,title,description,article_text,label
0,"NBA – Méconnaissable, Anthony Davis calciné pa...","Pour beaucoup, Anthony Davis est censé être le...","Cette saison, il semblait avoir véritablement ...",basket
1,Le voeu du président de l'Atlético Madrid conc...,Lionel Messi et Cristiano Ronaldo ont marqué l...,Cristiano Ronaldo et Lionel Messi n'ont jamai...,football
2,Pas de Ligue des champions ? Le Barça va jouer...,Le FC Barcelone connait les enjeux en ce qui c...,Le département espagnol de lutte contre la co...,football
3,Igor Tudor réagit à la polémique Matteo Guendouzi,"Ce vendredi, Igor Tudor a réagi en conférence ...","Ces derniers jours, des fuites dans la presse...",football
4,CLASSEMENT : Les meilleures - et les pires - s...,Depuis que le légendaire écossais a quitté le ...,Cela fait dix ans que Sir Alex Ferguson a bou...,football
...,...,...,...,...
945,NBA – Les images dingues du manoir à 31 millio...,Ultra-dominant quand il joue mais trop souvent...,"On peut le dire, il est compliqué de juger le ...",basket
946,NBA – Un proche de Michael Jordan : « J’ai dû ...,"Malgré son statut de légende vivante en NBA, L...","En compagnie des cuvées de draft 1984 et 1996,...",basket
947,"""Cristiano Ronaldo va salir sa légende à Al-Na...",Le transfert de Cristiano Ronaldo à Al-Nassr a...,Le quintuple Ballon d'Or a froncé les sourcil...,football
948,TRANSFERT. 27 essais en 27 capes : et si le pr...,"Meilleur marqueur de l’histoire de la Géorgie,...","L’an dernier, au moment d’apprendre qu’un cert...",rugby
